In [28]:
import pandas as pd
import prince
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel('train_data_cleaned.xlsx')

In [ ]:
# Definir variables categóricas
cat_vars = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Transported', 'CabinDeck', 'CabinSide']

# Limpieza rápida: Solo variables categóricas y eliminar nulos
df_mca = df[cat_vars].copy().astype(str)

print(f"Pasajeros analizados: {len(df_mca)}")
df_mca.head()

Pasajeros analizados: 8693


,HomePlanet,CryoSleep,Destination,VIP,Transported,CabinDeck,CabinSide
0,Europa,0,TRAPPIST-1e,0,0,B,P
1,Earth,0,TRAPPIST-1e,0,1,F,S
2,Europa,0,TRAPPIST-1e,1,0,A,S
3,Europa,0,TRAPPIST-1e,0,0,A,S
4,Earth,0,TRAPPIST-1e,0,1,F,S


In [30]:
# Inicializar y ajustar el MCA
mca = prince.MCA(
    n_components=2,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mca = mca.fit(df_mca)

In [31]:
# Obtener resumen de autovalores
eig = mca.eigenvalues_summary

plt.figure(figsize=(10, 4))
plt.bar(eig.index, eig['% of variance'], color='#3498db', alpha=0.8)
plt.plot(eig.index, eig['% of variance'], marker='o', color='#e74c3c')
plt.title('Varianza Explicada por cada Eje (Dimensiones)')
plt.xlabel('Dimensión')
plt.ylabel('% de Varianza')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print(f"Los dos primeros ejes explican el {eig['% of variance'].iloc[0:2].sum():.2f}% de la varianza.")

ValueError: Unknown format code 'f' for object of type 'str'

In [25]:
# Obtenemos las contribuciones de las columnas (categorías)
col_contributions = mca.column_contributions_
col_coordinates = mca.column_coordinates(df_acm)

# Unimos coordenadas y contribuciones para el análisis
analysis_df = col_coordinates.copy()
analysis_df.columns = ['Dim 0', 'Dim 1']
analysis_df['Contrib_Dim0'] = col_contributions[0]
analysis_df['Contrib_Dim1'] = col_contributions[1]

print("Top 10 categorías que más contribuyen a la Dimensión 0:")
print(analysis_df.sort_values(by='Contrib_Dim0', ascending=False)['Contrib_Dim0'].head(10))

Top 10 categorías que más contribuyen a la Dimensión 0:
HomePlanet__Europa          0.279115
CabinDeck__B                0.132299
CabinDeck__C                0.110806
HomePlanet__Earth           0.084417
Destination__55 Cancri e    0.082883
CabinDeck__F                0.048806
Transported__0              0.038872
Transported__1              0.038313
CabinDeck__A                0.034959
CabinDeck__G                0.033697
Name: Contrib_Dim0, dtype: float64


In [26]:
# Graficamos las categorías en el plano factorial
# prince tiene un método integrado, pero con Plotly es más legible
plot_data = analysis_df.reset_index()
plot_data.columns = ['Variable_Categoria', 'Dim 0', 'Dim 1', 'Contrib_0', 'Contrib_1']

fig_mca = px.scatter(
    plot_data, 
    x='Dim 0', 
    y='Dim 1', 
    text='Variable_Categoria',
    size='Contrib_0', # El tamaño indica cuánto aporta a la Dim 0
    color='Variable_Categoria',
    title='Mapa del ACM: Relación entre Categorías',
    labels={'Dim 0': 'Eje 1', 'Dim 1': 'Eje 2'}
)

fig_mca.update_traces(textposition='top center')
fig_mca.show()